# Análise de Churn — Telco Customer Churn

Resumo rápido
- Objetivo: identificar clientes com maior probabilidade de churn e definir um ponto de operação (threshold) que equilibre ação e custo para campanhas de retenção.
- Dataset: `WA_Fn-UseC_-Telco-Customer-Churn.csv` (dados de clientes de uma operadora de telecom).
- Resultado principal: usando LogisticRegression com `class_weight='balanced'` conseguimos elevar o recall para ~80% (trade-off com precisão).

Como usar este notebook
1. Carregamento & Limpeza — carrega os dados e aplica conversões básicas.
2. Análise Exploratória (EDA) — visualizações e estatísticas descritivas.
3. Modelagem — baselines (LogisticRegression, RandomForest) e comparação.
4. Tratamento de Desbalanceamento — class_weight e SMOTE testados.
5. Análise de Trade-off e Escolha do Threshold — curva precision-recall, seleção de thresholds e exemplo com matriz de confusão.
6. Conclusão e próximos passos.

Artefatos (na pasta `output/`)
- PR curve: `pr_curve_logreg_classweight.png`
- Tabela de thresholds: `pr_thresholds_logreg_classweight.csv`
- Modelos salvos: `LogisticRegression_class_weight.joblib`, `RandomForest_smote.joblib`, etc.
- Resumos: `model_comparison_balance.csv`, `selected_thresholds_summary.csv`

Nota: algumas células incluem saídas estáticas (imagens e relatórios já gerados) para visualização imediata sem necessidade de executar todo o notebook.

In [ ]:
# Carregamento e limpeza inicial
# Aqui carregamos o CSV e aplicamos conversões necessárias (TotalCharges -> num, remover customerID)
import pandas as pd
import numpy as np

DATA_PATH = r"c:\Users\Gabriel\Downloads\LLM\WA_Fn-UseC_-Telco-Customer-Churn.csv"

df = pd.read_csv(DATA_PATH)
print('Shape original:', df.shape)

# Converter TotalCharges para numérico (algumas linhas estão vazias)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
print('TotalCharges NaN before drop:', df['TotalCharges'].isnull().sum())

# Remover linhas com TotalCharges NaN
df = df.dropna(subset=['TotalCharges']).copy()

# Ajustes de tipos e limpeza simples
df['SeniorCitizen'] = df['SeniorCitizen'].astype('category')
# customerID não é usado como feature
if 'customerID' in df.columns:
    df = df.drop(columns=['customerID'])

# Normalizar valores "No internet service" e "No phone service" para variáveis booleanas
cols_replace_no = ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','MultipleLines']
for c in cols_replace_no:
    if c in df.columns:
        df[c] = df[c].replace({'No internet service':'No', 'No phone service':'No'})

print('Shape after cleaning:', df.shape)
df.head()

In [ ]:
# Modelagem baseline e comparação
# Treinamos dois modelos simples: LogisticRegression e RandomForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
import joblib
import sklearn

# separar X/y
y = df['Churn'].map({'Yes':1,'No':0})
X = df.drop(columns=['Churn'])

# colunas numéricas e categóricas
num_cols = ['tenure','MonthlyCharges','TotalCharges']
cat_cols = [c for c in X.columns if c not in num_cols]

# compatibiliza OneHotEncoder conforme a versão do sklearn
v = sklearn.__version__.split('.')
major, minor = int(v[0]), int(v[1]) if len(v)>1 else 0
if major > 1 or (major == 1 and minor >= 2):
    from sklearn.preprocessing import OneHotEncoder as OHE
    ohe = OHE(handle_unknown='ignore', sparse_output=False)
else:
    from sklearn.preprocessing import OneHotEncoder as OHE
    ohe = OHE(handle_unknown='ignore', sparse=False)

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', ohe, cat_cols)
])

# Split para treino/teste
a = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=a)

# LogisticRegression com class_weight='balanced' (bom para recall)
log_pipe = Pipeline([
    ('pre', preprocessor),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
])
log_pipe.fit(X_train, y_train)
pred_log = log_pipe.predict(X_test)
probs_log = log_pipe.predict_proba(X_test)[:,1]
print('LogisticRegression (class_weight)')
print(classification_report(y_test, pred_log))
print('ROC AUC:', roc_auc_score(y_test, probs_log))
joblib.dump(log_pipe, 'output/LogisticRegression_class_weight.joblib')

# RandomForest baseline (pode ser usado com SMOTE)
rf_pipe = Pipeline([
    ('pre', preprocessor),
    ('clf', RandomForestClassifier(n_estimators=200, random_state=42))
])
rf_pipe.fit(X_train, y_train)
pred_rf = rf_pipe.predict(X_test)
probs_rf = rf_pipe.predict_proba(X_test)[:,1]
print('RandomForest')
print(classification_report(y_test, pred_rf))
print('ROC AUC:', roc_auc_score(y_test, probs_rf))
joblib.dump(rf_pipe, 'output/RandomForest_baseline.joblib')

print('Pipelines salvos em output/')

## Análise de Trade-off e Escolha do Threshold

Nesta seção vamos analisar o trade-off entre precisão e recall para o modelo LogisticRegression com `class_weight='balanced'`. Usamos a curva Precision-Recall e uma tabela de thresholds previamente gerada para escolher pontos de operação.

Passos:
- Exibir curva Precision-Recall
- Explicar implicações de negócio do trade-off
- Sugerir 3 thresholds operacionais: alto recall, balanceado e alta precisão
- Para o threshold balanceado: calcular matriz de confusão e métricas finais

Os artefatos utilizados (gerados anteriormente):
- `output/pr_curve_logreg_classweight.png`
- `output/pr_thresholds_logreg_classweight.csv`
- `output/LogisticRegression_class_weight.joblib` (pipeline salvo)


In [ ]:
# Análise de trade-off: carregar artefatos gerados (PR curve e tabela de thresholds)
import pandas as pd
from IPython.display import Image, display
import joblib

OUT_DIR = r"c:\Users\Gabriel\Downloads\LLM\output"

# carregar thresholds
th = pd.read_csv(OUT_DIR + r"\pr_thresholds_logreg_classweight.csv")
print('Thresholds (exemplo - head):')
print(th.head())

# exibir PR curve gerada
display(Image(filename=OUT_DIR + r"\pr_curve_logreg_classweight.png"))

# carregar pipeline log
pipe = joblib.load(OUT_DIR + r"\LogisticRegression_class_weight.joblib")

# carregar dataset limpo e probabilidades para análise (mostraremos métricas em célula posterior)
df = pd.read_csv(r"c:\Users\Gabriel\Downloads\LLM\WA_Fn-UseC_-Telco-Customer-Churn.csv")
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna(subset=['TotalCharges']).copy()
y = df['Churn'].map({'Yes':1,'No':0})
X = df.drop(columns=['Churn','customerID'], errors='ignore')
probs = pipe.predict_proba(X)[:,1]
print('Probabilidades calculadas para todo o dataset (usadas para análise de thresholds)')


In [ ]:
### Escolha de 3 pontos de operação (thresholds) e exemplo prático

A partir da tabela de thresholds gerada, propomos três pontos de operação:

- Alto recall (prioriza encontrar a maior parte dos churners).
- Balanceado (compromisso entre precisão e recall, otimiza F1).
- Alta precisão (evitar falsos positivos, útil quando custo de ação é alto).

Os valores calculados automaticamente foram (resumo):

- Alto recall — threshold = 0.005816 — precision ≈ 0.266, recall = 1.000, F1 ≈ 0.420
- Balanceado — threshold = 0.549243 — precision ≈ 0.538, recall ≈ 0.788, F1 ≈ 0.640
- Alta precisão — threshold = 1.000000 — precision = 1.000, recall = 0.000

Matriz de confusão para o threshold 'balanceado' (threshold = 0.549243):

```
[[3898, 1265],
 [ 396, 1473]]
```

Classification report (balanceado):

```
              precision    recall  f1-score   support

           0     0.9078    0.7550    0.8244      5163
           1     0.5380    0.7881    0.6395      1869

    accuracy                         0.7638      7032
   macro avg     0.7229    0.7716    0.7319      7032
weighted avg     0.8095    0.7638    0.7752      7032
```

Interpretação de negócio resumida:
- O threshold balanceado captura ~79% dos churners (bom para campanhas de retenção), com precisão aceitável (~54%).
- Se há capacidade para validar mais clientes, reduzir o threshold aumenta recall mas aumenta custo por falsos positivos.
- Próximo passo: testar campanhas piloto com o threshold balanceado e medir taxa de conversão de ações de retenção para avaliar custo-benefício.


### Escolha de 3 pontos de operação (thresholds) e exemplo prático

A partir da tabela de thresholds gerada, sugerimos três pontos de operação:

1. Alto recall (prioriza encontrar a maior parte dos churners)
   - threshold = 0.005816
   - precision ≈ 0.2658, recall = 1.0000, F1 ≈ 0.4200
   - Observação: identifica todos os churners mas gera muitos falsos positivos.

2. Balanceado (maximiza F1 aproximado)
   - threshold = 0.549243
   - precision ≈ 0.5380, recall ≈ 0.7881, F1 ≈ 0.6395

3. Alta precisão (prioriza evitar falsos positivos)
   - threshold = 1.000000
   - precision = 1.0, recall = 0.0, F1 = 0.0
   - Observação: threshold=1 significa classificar como churn apenas se P=1; prático somente como referência extrema.

Abaixo está a matriz de confusão e o classification report para o threshold 'balanceado' (threshold = 0.549243):

Confusion matrix (linhas=verdadeiro, colunas=predito):

[[3898, 1265],
 [ 396, 1473]]

Classification report (balanceado):

precision recall f1-score support

0 0.9078 0.7550 0.8244 5163
1 0.5380 0.7881 0.6395 1869

accuracy 0.7638 7032
macro avg 0.7229 0.7716 0.7319 7032
weighted avg 0.8095 0.7638 0.7752 7032

A curva Precision-Recall também está incluída acima para referência visual.

Interpretação de negócio rápida:
- Se o objetivo é reduzir churn proativamente, geralmente priorizamos recall (capturar a maior parcela possível de churners). O ponto 'balanceado' oferece um bom compromisso: captura ~78.8% dos churners enquanto mantém precisão em ~53.8%.
- O ponto 'alto recall' garante quase nenhum churn perdido, mas exigiria validar muitas ações (alto custo por falsos positivos).
- Recomendação: começar com o threshold 'balanceado' e ajustar conforme capacidade operacional para lidar com falsos positivos; para campanhas agressivas, mover para um threshold menor (aumentando recall).

## Minhas Conclusões e Próximos Passos

    > Conclusões Pessoais
No início do projeto, o primeiro modelo que treinei apresentou uma acurácia de 80%, o que inicialmente parecia um ótimo resultado. No entanto, o que eu aprendi foi que uma única métrica, como a acurácia, pode ser enganosa, especialmente em problemas de negócio como este.

 A lição mais importante para mim foi quando analisei a métrica de Recall e percebi que o modelo estava falhando em identificar 43% dos clientes que realmente cancelariam. Entendi que, para a empresa, era muito mais valioso identificar corretamente um cliente em risco (alto Recall) do que apenas acertar a previsão geral. Foi nesse momento que o foco do projeto mudou de "criar um modelo preciso" para "criar um modelo útil".

A decisão de aplicar a técnica de class_weight para tratar o desbalanceamento dos dados foi o ponto de virada. Ao fazer isso, consegui elevar o Recall para 80%, transformando o modelo de uma curiosidade estatística para uma ferramenta de negócio realmente útil, capaz de guiar ações proativas de retenção e potencialmente salvar uma receita significativa para a empresa.

    > Próximos Passos
Monitoramento e Operacionalização: O próximo passo natural seria implementar este modelo em um ambiente de produção. Uma sugestão seria criar um dashboard (em Power BI, por exemplo) que consumisse as previsões do modelo e mostrasse para a equipe de negócio, de forma diária ou semanal, a lista de clientes com maior risco de churn.

Colaboração Interdepartamental: Com essa lista em mãos, eu recomendaria trabalhar diretamente com a equipe de Marketing para desenhar e executar campanhas de retenção direcionadas, como oferecer um desconto ou um benefício específico para os clientes identificados pelo modelo.

Melhoria Contínua: Por fim, seria crucial monitorar a performance do modelo ao longo do tempo (uma prática conhecida como MLOps) para garantir que ele continue preciso à medida que novos dados chegam, e re-treiná-lo periodicamente para capturar novas tendências de comportamento dos clientes.